# 使用数据集

在之前的实验中，你使用数据存 *储提供基* 于云的集中式数据访问。在本实验中，你将了解 *数据集*，这种进一步的抽象可以更轻松地使用特定数据进行试验和训练。

## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **注意**： 如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备数据

在上一实验中，你创建了数据存储。数据集通常（尽管并非总是）基于数据存储中的数据。

如果未完成上一实验，请运行以下代码将两个本地 CSV 文件上传到工作区中的默认数据存储（如果已完成上一实验，则会直接覆盖相同文件）。

In [ ]:
ws.get_default_datastore().upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 将糖尿病 csv 文件上传到 /data 中
                       target_path='diabetes-data/', # 将其放在数据存储的文件夹路径中
                       overwrite=True, # 替换名称相同的现有文件
                       show_progress=True)

## 创建表格数据集

数据集是封装特定数据源的对象。接下来根据上传到数据存储的糖尿病数据创建数据集，然后查看前 20 条记录。这种情况下，数据在 CSV 文件中为结构化格式，因此我们将使用 *表格* 数据集。

In [ ]:
from azureml.core import Dataset

# 获取默认数据存储
default_ds = ws.get_default_datastore()

# 从数据存储上的路径创建表格数据集（这可能需要一些时间）
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 将前 20 行显示为 Pandas 数据帧
tab_data_set.take(20).to_pandas_dataframe()

如上述代码中所见，可以轻松地将表格数据集转换为 Pandas 数据帧，从而使用常见的 python 技术处理数据。

## 创建文件数据集

你创建的数据集是 *表格* 数据集，可以在数据集定义所包含的结构化文件中作为包含所有数据的数据帧读取。这对于表格数据非常有效，但在某些机器学习场景中，可能需要使用非结构化数据；或者你可能只想通过自己的代码读取文件中的数据。为此，可以使用 *文件* 数据集，该数据集在虚拟装入点创建文件路径列表，用于读取文件中的数据。

In [ ]:
# 通过数据存储上的路径创建文件数据集（这可能需要一些时间）
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# 获取数据集中的文件
for file_path in file_data_set.to_path():
    print(file_path)

## 注册数据集

创建引用糖尿病数据的数据集后，可以将其注册，确保工作区中运行的所有试验可轻松对其进行访问。

我们将表格数据集注册为 **“糖尿病数据集”**，将文件数据集注册为 **“糖尿病文件”**。

In [ ]:
# 注册表格数据集
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

# 注册文件数据集
file_data_set = file_data_set.register(workspace=ws, 
                           name='diabetes file dataset',
                           description='diabetes files',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Datasets registered')

在 [Azure 机器学习工作室](https://ml.azure.com)中的工作区的 **“数据集”** 页上可以查看和管理数据集。你还可以从工作区对象获取数据集列表：

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

如果已完成实验 2A 和 2B，你将看到注册的数据集包含使用可视化设计器工具创建的转换。你可能还会注意到，以上一练习中使用 *“工作室”* 界面创建的数据集名称注册 **糖尿病数据集时**，会创建新 *版本* 的数据集。通过对数据集进行版本控制，可以重新定义数据集，从而无需破坏依赖先前定义的现有试验或管道。默认返回最新版本的已命名数据集，但可以通过指定版本号检索特定版本的数据集，如下所示：

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


## 从表格数据集训练模型

有数据集后，即可开始从中训练模型。可以在运行脚本的估算器中将数据集作为*输入*传递给脚本。

运行以下两个代码单元格，创建以下内容：

1.名为 **diabetes_training_from_tab_dataset** 的文件夹
2.使用表格数据集（作为 *输入* 传递给脚本）训练分类模型的脚本。

In [ ]:
import os

# 为试验文件创建文件夹
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 设置正则化超参数（作为参数传递给脚本）
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 获取试验运行上下文
run = Run.get_context()

# 加载糖尿病数据（作为输入数据集传递）
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练逻辑回归模型
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 注意，保存在输出文件夹中的文件会自动上传到试验记录
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

现在可以创建估算器来运行脚本，并为训练数据集定义由脚本读取的命名 *输入*。

> **注意**： **Dataset** 类在 **azureml-dataprep** 包（随 SDK 一起安装）中定义，且该包包含对 **pandas** 的可选支持（由 **to_pandas_dataframe()** 方法采用），因此需要在要运行训练试验的环境中包含此包。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# 设置脚本参数
script_params = {
    '--regularization': 0.1
}

# 获取训练数据集
diabetes_ds = ws.datasets.get("diabetes dataset")

# 创建估算器
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes')], # 将数据集对象作为输入传递...
                    pip_packages=['azureml-dataprep[pandas]'] # ...因此我们需要 dataprep 包
                   )

# 创建试验
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 运行试验
run = experiment.submit(config=estimator)
# 在运行时显示运行详细信息
RunDetails(run).show()
run.wait_for_completion()

首次运行试验时，可能需要一些时间来设置 Python 环境 - 后续运行会更快。

试验完成后，在小组件中查看 **azureml-logs/70_driver_log.txt** 输出日志和运行所生成的指标。

与所有试验一样，可以在 [Azure 机器学习工作室](https://ml.azure.com)中查看试验运行的详细信息，还可以编写代码检索生成的指标和文件：

In [ ]:
# 获取记录的指标
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

训练的模型另存为 **输出** 文件夹中的 **diabetes_model.pkl** 文件，因此可以注册它。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (tabular dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 从文件数据集训练模型

你已了解了如何使用*表格*数据集中的训练数据来训练模型。但*文件*数据集呢？

使用文件数据集时，传递给脚本的数据集输入表示包含文件路径的装入点。从这些文件中读取数据的方式取决于文件中的数据类型及其预期用途。对于糖尿病 CSV 文件，可以使用 Python **glob** 模块在数据集定义的虚拟装入点中创建文件列表，并将其全部读入可联结为单个数据帧的 Pandas 数据帧中。

运行以下两个代码单元格，创建以下内容：

1.名为 **diabetes_training_from_file_dataset** 的文件夹
2.使用文件数据集（作为 *输入* 传递给脚本）训练分类模型的脚本。

In [ ]:
import os

# 为试验文件创建文件夹
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import argparse
from azureml.core import Workspace, Dataset, Experiment, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# 设置正则化超参数（作为参数传递给脚本）
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 获取试验运行上下文
run = Run.get_context()

# 加载糖尿病数据集
print("Loading Data...")
data_path = run.input_datasets['diabetes'] # 从估算器输入中获取训练数据
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files))

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练逻辑回归模型
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 注意，保存在输出文件夹中的文件会自动上传到试验记录
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

接下来需要更改将数据集传递给估算器的方式 - 这需要定义脚本可以从中读取文件的装入点。对于大量数据，通常使用 **as_mount** 方法直接从数据集源流式处理文件；但在本地计算中运行时（如本例所示），需要使用 **as_download** 选项将数据集文件下载到本地文件夹。

另外，由于 **Dataset** 类在 **azureml-dataprep** 包中定义，因此需要将其包含在试验环境中。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# 设置脚本参数
script_params = {
    '--regularization': 0.1
}

# 获取训练数据集
diabetes_ds = ws.datasets.get("diabetes file dataset")

# 创建估算器
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes').as_download(path_on_compute='diabetes_data')], # 将数据集对象作为输入传递
                    pip_packages=['azureml-dataprep[pandas]'] # 因此我们需要 dataprep 包
                   )

# 创建试验
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 运行试验
run = experiment.submit(config=estimator)
# 在运行时显示运行详细信息
RunDetails(run).show()
run.wait_for_completion()

试验完成后，在小组件中查看 **azureml-logs/70_driver_log.txt** 输出日志，以验证文件数据集是否已处理以及数据文件是否已下载。

与所有试验一样，可以在 [Azure 机器学习工作室](https://ml.azure.com)中查看试验运行的详细信息，还可以编写代码检索生成的指标和文件：

In [ ]:
# 获取记录的指标
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

同样，注册训练的模型。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (file dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **更多信息**： 有关使用数据集进行训练的更多信息，请参阅 Azure ML 文档中的[使用数据集进行训练](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)。